In [ ]:
# Install dependencies
!pip install streamlit pyngrok mediapipe transformers torch torchvision numpy opencv-python-headless ffmpeg-python
!apt-get install ffmpeg -y

# Install ngrok
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvzf ngrok-v3-stable-linux-amd64.tgz
!mv ngrok /usr/local/bin/

# Install required packages
!pip install mediapipe transformers torch torchvision numpy opencv-python-headless ffmpeg-python
!apt-get install ffmpeg -y

# Note: You need an ngrok auth token. Sign up at https://ngrok.com, get your token from https://dashboard.ngrok.com/get-started/your-authtoken
# Replace 'YOUR_NGROK_AUTH_TOKEN' with your actual token
!ngrok authtoken 305NarI0dy6eL1ti16UaQvaQEzE_7qSCpuDQ6DsH7JCxBTzXg

In [ ]:
# Mount Google Drive
from google.colab import drive
import os
import json
import logging

#drive.mount('/content/drive', force_remount=True)

# Specify Google Drive output path
DRIVE_OUTPUT_PATH = '/content/drive/MyDrive/Surveillance_Output'  # CHANGE THIS TO YOUR DESIRED PATH
os.makedirs(DRIVE_OUTPUT_PATH, exist_ok=True)

# Define Streamlit app script
streamlit_script = """
import streamlit as st
import os
import json
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Streamlit app configuration
st.set_page_config(page_title="Enhanced Retail Surveillance", layout="wide")
st.title("Enhanced Retail Surveillance System")
st.write("Upload a video to analyze behavior using VideoMAE.")

# Video upload
uploaded_file = st.file_uploader("Choose a video file", type=["mp4", "webm", "avi", "mov"])
if uploaded_file is not None:
    # Save uploaded video to /content/
    input_video_path = os.path.join("/content", uploaded_file.name)
    try:
        with open(input_video_path, "wb") as f:
            f.write(uploaded_file.getvalue())
        st.success(f"✅ Video uploaded: {uploaded_file.name}")
        # Write processing flag
        process_flag = "/content/process_video.txt"
        with open(process_flag, "w") as f:
            f.write(input_video_path)
        st.info("Processing video... Re-run Cell 3 in Colab, then refresh this page to see results.")
    except Exception as e:
        st.error(f"Error saving video: {str(e)}")

# Display results
result_path = "/content/drive/MyDrive/Surveillance_Output/result.json"
process_flag = "/content/process_video.txt"
video_file_name = "unknown_video"
if os.path.exists(process_flag):
    try:
        with open(process_flag, "r") as f:
            input_video_path = f.read().strip()
            video_file_name = os.path.basename(input_video_path)
    except Exception as e:
        logging.error(f"❌ Error reading process flag: {str(e)}")
if os.path.exists(result_path):
    try:
        with open(result_path, "r") as f:
            result_data = json.load(f)
        violence_detected = result_data.get("violence_detected", 0)
        result_text = "Violence Detected" if violence_detected >= 1 else "Normal"
        formatted_result = f"{result_text}"
        st.subheader(formatted_result)
    except Exception as e:
        st.error(f"Error loading result: {str(e)}")
        logging.error(f"❌ Error loading result.json: {str(e)}")
else:
    st.info("No video processed yet. Upload a video to start.")
"""

# Save Streamlit script
try:
    with open('/content/surveillance_streamlit_app.py', 'w') as f:
        f.write(streamlit_script)
    print(f"✅ Saved Streamlit app to /content/surveillance_streamlit_app.py")
except Exception as e:
    print(f"❌ Error saving Streamlit script: {str(e)}")

In [ ]:


# Imports
from google.colab import drive
import cv2
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from transformers import VideoMAEForVideoClassification, VideoMAEImageProcessor
import torchvision.transforms as T
from collections import deque
from scipy.optimize import linear_sum_assignment
import time
from datetime import datetime
import warnings
import logging
import json
import os
import shutil

warnings.filterwarnings('ignore')
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Mount Google Drive
#drive.mount('/content/drive', force_remount=True)

# Specify Google Drive output path
DRIVE_OUTPUT_PATH = '/content/drive/MyDrive/Surveillance_Output'  # CHANGE THIS TO MATCH YOUR PATH
os.makedirs(DRIVE_OUTPUT_PATH, exist_ok=True)

# Surveillance processing classes
class PersonTracker:
    """Enhanced person tracker with improved detection and tracking"""
    def __init__(self, max_disappeared=30, max_distance=100):
        self.next_id = 0
        self.objects = {}
        self.disappeared = {}
        self.max_disappeared = max_disappeared
        self.max_distance = max_distance

    def register(self, centroid, bbox):
        self.objects[self.next_id] = {
            'centroid': centroid,
            'bbox': bbox,
            'positions': deque([centroid], maxlen=50),
            'behavior_history': deque(maxlen=10),
            'last_behavior': 'Normal',
            'behavior_score': 0.0,
            'frame_count': 0
        }
        self.disappeared[self.next_id] = 0
        self.next_id += 1

    def deregister(self, object_id):
        del self.objects[object_id]
        del self.disappeared[object_id]

    def update(self, detections):
        if len(detections) == 0:
            for object_id in list(self.disappeared.keys()):
                self.disappeared[object_id] += 1
                if self.disappeared[object_id] > self.max_disappeared:
                    self.deregister(object_id)
            return self.objects

        input_centroids = np.array([det['centroid'] for det in detections])
        input_bboxes = [det['bbox'] for det in detections]

        if len(self.objects) == 0:
            for i, detection in enumerate(detections):
                self.register(input_centroids[i], input_bboxes[i])
        else:
            object_centroids = np.array([obj['centroid'] for obj in self.objects.values()])
            object_ids = list(self.objects.keys())
            D = np.linalg.norm(object_centroids[:, np.newaxis] - input_centroids, axis=2)

            if D.shape[0] <= D.shape[1]:
                rows, cols = linear_sum_assignment(D)
            else:
                rows, cols = linear_sum_assignment(D.T)
                rows, cols = cols, rows

            used_row_indices = set()
            used_col_indices = set()

            for (row, col) in zip(rows, cols):
                if D[row, col] <= self.max_distance:
                    object_id = object_ids[row]
                    self.objects[object_id]['centroid'] = input_centroids[col]
                    self.objects[object_id]['bbox'] = input_bboxes[col]
                    self.objects[object_id]['positions'].append(input_centroids[col])
                    self.objects[object_id]['frame_count'] += 1
                    self.disappeared[object_id] = 0
                    used_row_indices.add(row)
                    used_col_indices.add(col)

            unused_row_indices = set(range(0, D.shape[0])).difference(used_row_indices)
            unused_col_indices = set(range(0, D.shape[1])).difference(used_col_indices)

            for row in unused_row_indices:
                object_id = object_ids[row]
                self.disappeared[object_id] += 1
                if self.disappeared[object_id] > self.max_disappeared:
                    self.deregister(object_id)

            for col in unused_col_indices:
                self.register(input_centroids[col], input_bboxes[col])

        return self.objects

class EnhancedRetailSurveillanceSystem:
    def __init__(self, drive_output_path=DRIVE_OUTPUT_PATH):
        print("🏪 Initializing Enhanced Retail Surveillance System with VideoMAE...")
        self.drive_output_path = drive_output_path
        try:
            os.makedirs(self.drive_output_path, exist_ok=True)
            print(f"✅ Created/Verified Google Drive output directory: {self.drive_output_path}")
        except Exception as e:
            print(f"❌ Error creating Google Drive directory {self.drive_output_path}: {e}")
            raise
        self.setup_models()
        self.setup_detection_parameters()
        self.frame_buffer = {}
        self.buffer_size = 16  # Match reference code
        self.person_tracker = PersonTracker(max_disappeared=30, max_distance=100)
        self.setup_person_detection()
        self.bg_subtractor = cv2.createBackgroundSubtractorMOG2(
            history=500, varThreshold=16, detectShadows=True
        )
        self.activity_log = []
        self.violence_alerts = []
        self.detection_stats = {
            'total_frames': 0,
            'violence_detected': 0,
            'abnormal_behavior': 0,
            'normal_behavior': 0
        }
        print("✅ Enhanced Retail Surveillance System initialized!")

    def setup_models(self):
        print("🤖 Setting up AI models...")
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if self.device.type == "cpu":
            print("⚠️ No GPU available, processing will be slower")
        self.violence_model = VideoMAEForVideoClassification.from_pretrained(
            "MCG-NJU/videomae-base-finetuned-kinetics"
        ).to(self.device)
        self.violence_model.eval()
        self.image_processor = VideoMAEImageProcessor.from_pretrained(
            "MCG-NJU/videomae-base-finetuned-kinetics", use_fast=True
        )
        self.behavior_classes = ['Normal', 'Suspicious', 'Aggressive', 'Violence', 'Theft']
        self.class_mapping = self.create_action_mapping()
        self.behavior_thresholds = {
            'Normal': 0.4,
            'Suspicious': 0.4,
            'Aggressive': 0.4,
            'Violence': 0.6,
            'Theft': 0.4
        }

    def create_action_mapping(self):
        """Map Kinetics-400 classes to 5 target classes"""
        kinetics_labels = self.violence_model.config.id2label
        mapping = {}
        for idx, label in kinetics_labels.items():
            label_lower = label.lower()
            if any(keyword in label_lower for keyword in ['fight', 'punch', 'kick', 'attack', 'hit']):
                mapping[idx] = 'Violence'
            elif any(keyword in label_lower for keyword in ['run away', 'sneak', 'hide']):
                mapping[idx] = 'Suspicious'
            elif any(keyword in label_lower for keyword in ['push', 'shout', 'argue', 'confront']):
                mapping[idx] = 'Aggressive'
            elif any(keyword in label_lower for keyword in ['steal', 'shoplift', 'pickpocket']):
                mapping[idx] = 'Theft'
            else:
                mapping[idx] = 'Normal'
        return mapping

    def setup_person_detection(self):
        """Setup person detection using HOG"""
        self.hog = cv2.HOGDescriptor()
        self.hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
        try:
            self.face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
            self.has_face_detection = True
        except:
            self.has_face_detection = False
            print("⚠️ Face detection not available")

    def setup_detection_parameters(self):
        """Setup detection parameters"""
        self.min_person_area = 2000
        self.max_person_area = 50000
        self.min_aspect_ratio = 0.3
        self.max_aspect_ratio = 3.0
        self.behavior_window = 10
        self.violence_threshold = 0.6
        self.suspicious_threshold = 0.4

    def detect_people(self, frame):
        """Detect people in the frame using HOG"""
        detections = []
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        try:
            (rects, weights) = self.hog.detectMultiScale(
                gray, winStride=(4, 4), padding=(8, 8), scale=1.05, hitThreshold=0.5
            )
            for i, (x, y, w, h) in enumerate(rects):
                area = w * h
                aspect_ratio = h / w if w > 0 else 0
                if (self.min_person_area <= area <= self.max_person_area and
                    self.min_aspect_ratio <= aspect_ratio <= self.max_aspect_ratio):
                    confidence = weights[i] if i < len(weights) else 0.5
                    detections.append({
                        'bbox': [x, y, w, h],
                        'centroid': (x + w//2, y + h//2),
                        'confidence': confidence,
                        'method': 'HOG'
                    })
        except Exception as e:
            print(f"HOG detection error: {e}")
        if len(detections) == 0:
            detections.extend(self.detect_motion_based_people(frame))
        detections = self.remove_overlapping_detections(detections)
        return detections

    def detect_motion_based_people(self, frame):
        """Detect people using motion analysis"""
        detections = []
        fg_mask = self.bg_subtractor.apply(frame)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)
        fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
        contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for contour in contours:
            area = cv2.contourArea(contour)
            if area > self.min_person_area:
                x, y, w, h = cv2.boundingRect(contour)
                aspect_ratio = h / w if w > 0 else 0
                if (self.min_aspect_ratio <= aspect_ratio <= self.max_aspect_ratio and
                    area <= self.max_person_area):
                    detections.append({
                        'bbox': [x, y, w, h],
                        'centroid': (x + w//2, y + h//2),
                        'confidence': 0.4,
                        'method': 'Motion'
                    })
        return detections

    def remove_overlapping_detections(self, detections, overlap_threshold=0.5):
        """Remove overlapping detections using NMS"""
        if len(detections) <= 1:
            return detections
        boxes = [[x, y, x+w, y+h] for x, y, w, h in [det['bbox'] for det in detections]]
        confidences = [det['confidence'] for det in detections]
        boxes = np.array(boxes)
        confidences = np.array(confidences)
        indices = cv2.dnn.NMSBoxes(boxes.tolist(), confidences.tolist(), 0.3, overlap_threshold)
        if len(indices) > 0:
            indices = indices.flatten()
            return [detections[i] for i in indices]
        return []

    def analyze_behavior(self, frame, person_bbox, person_id):
        """Analyze behavior for a person region using a sequence of frames"""
        try:
            x, y, w, h = person_bbox
            padding = 20
            x1 = max(0, x - padding)
            y1 = max(0, y - padding)
            x2 = min(frame.shape[1], x + w + padding)
            y2 = min(frame.shape[0], y + h + padding)
            person_region = frame[x1:x2, y1:y2]  # Corrected order to match frame indexing
            if person_region.size == 0 or person_region.shape[0] < 10 or person_region.shape[1] < 10:
                logging.warning(f"Person {person_id}: Invalid region (shape: {person_region.shape})")
                return {'behavior': 'Normal', 'confidence': 0.5, 'score': 0.0}

            person_region = cv2.cvtColor(person_region, cv2.COLOR_BGR2RGB)
            pil_image = Image.fromarray(person_region)

            if person_id not in self.frame_buffer:
                self.frame_buffer[person_id] = deque(maxlen=self.buffer_size)
            self.frame_buffer[person_id].append(pil_image)

            if len(self.frame_buffer[person_id]) < self.buffer_size:
                logging.info(f"Person {person_id}: Waiting for {self.buffer_size} frames, currently {len(self.frame_buffer[person_id])}")
                return {'behavior': 'Normal', 'confidence': 0.5, 'score': 0.0}

            # Ensure all frames are resized to 224x224
            resized_frames = []
            for img in self.frame_buffer[person_id]:
                img_resized = img.resize((224, 224), Image.Resampling.LANCZOS)
                resized_frames.append(img_resized)

            inputs = self.image_processor(
                resized_frames, return_tensors="pt", num_frames=self.buffer_size
            )
            input_tensor = inputs['pixel_values'].to(self.device)
            logging.info(f"Person {person_id}: Input tensor shape: {input_tensor.shape}")
            with torch.no_grad():
                outputs = self.violence_model(input_tensor)
                logits = outputs.logits
                behavior_prob = torch.nn.functional.softmax(logits, dim=1)
                behavior_pred = torch.argmax(behavior_prob, dim=1).item()
                behavior_score = behavior_prob[0, behavior_pred].item()
                predicted_behavior = self.class_mapping.get(behavior_pred, 'Normal')
                logging.info(f"Person {person_id}: Predicted {predicted_behavior} (score: {behavior_score:.2f})")
                if behavior_score < 0.3:
                    predicted_behavior = 'Normal'
                    behavior_score = 0.3
                return {
                    'behavior': predicted_behavior,
                    'confidence': behavior_score,
                    'score': behavior_score
                }
        except Exception as e:
            logging.error(f"Person {person_id}: Behavior analysis error: {e}")
            return {'behavior': 'Normal', 'confidence': 0.5, 'score': 0.0}

    def update_behavior_history(self, person_id, behavior_result):
        """Update behavior history for a person"""
        if person_id in self.person_tracker.objects:
            person = self.person_tracker.objects[person_id]
            person['behavior_history'].append(behavior_result)
            if len(person['behavior_history']) >= 3:
                recent_behaviors = list(person['behavior_history'])[-5:]
                behavior_counts = {}
                for behavior in recent_behaviors:
                    behavior_type = behavior['behavior']
                    behavior_counts[behavior_type] = behavior_counts.get(behavior_type, 0) + 1
                dominant_behavior = max(behavior_counts, key=behavior_counts.get)
                person['last_behavior'] = dominant_behavior
                person['behavior_score'] = np.mean([b['score'] for b in recent_behaviors])

    def draw_person_annotations(self, frame, tracked_people):
        """Draw bounding boxes and behavior labels for tracked people"""
        annotated_frame = frame.copy()
        for person_id, person_data in tracked_people.items():
            x, y, w, h = person_data['bbox']
            behavior = person_data['last_behavior']
            behavior_score = person_data['behavior_score']
            color = {
                'Violence': (0, 0, 255),
                'Aggressive': (0, 100, 255),
                'Suspicious': (0, 255, 255),
                'Theft': (255, 0, 255),
                'Normal': (0, 255, 0)
            }.get(behavior, (0, 255, 0))
            thickness = 3 if behavior in ['Violence', 'Aggressive', 'Theft'] else 2
            cv2.rectangle(annotated_frame, (x, y), (x+w, y+h), color, thickness)
            label = f"{behavior} ({behavior_score:.2f})" if behavior_score > 0 else behavior
            font_scale = 0.7
            font_thickness = 2
            (text_width, text_height), baseline = cv2.getTextSize(
                label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness
            )
            cv2.rectangle(annotated_frame,
                         (x, y - text_height - 10),
                         (x + text_width, y),
                         color, -1)
            cv2.putText(annotated_frame, label, (x, y - 5),
                       cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), font_thickness)
            cv2.putText(annotated_frame, f"ID:{person_id}", (x, y + h + 20),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        return annotated_frame

    def process_frame(self, frame, frame_count):
        """Process a single frame for person detection and behavior analysis"""
        detections = self.detect_people(frame)
        tracked_people = self.person_tracker.update(detections)
        alerts = []

        for person_id, person_data in tracked_people.items():
            behavior_result = self.analyze_behavior(frame, person_data['bbox'], person_id)
            self.update_behavior_history(person_id, behavior_result)
            if behavior_result['behavior'] in ['Violence', 'Aggressive', 'Theft']:
                alerts.append({
                    'type': behavior_result['behavior'],
                    'person_id': person_id,
                    'frame': frame_count,
                    'confidence': behavior_result['confidence']
                })
                if behavior_result['behavior'] == 'Violence':
                    self.detection_stats['violence_detected'] += 1
                self.detection_stats['abnormal_behavior'] += 1
            else:
                self.detection_stats['normal_behavior'] += 1

        annotated_frame = self.draw_person_annotations(frame, tracked_people)
        return annotated_frame, {'alerts': alerts}

    def process_video(self, video_source=0, output_video=None, max_frames=100, display_frames=True, colab_mode=True):
        """Main video processing loop"""
        print(f"🎥 Starting video processing from source: {video_source}")
        if isinstance(video_source, str) and video_source.endswith('.webm'):
            temp_path = os.path.join(self.drive_output_path, 'temp.mp4')
            os.system(f'ffmpeg -i {video_source} -c:v libx264 -c:a aac {temp_path}')
            if os.path.exists(temp_path):
                video_source = temp_path
                print(f"✅ Converted .webm to .mp4: {video_source}")
            else:
                print("❌ Failed to convert .webm to .mp4")
                return False
        cap = cv2.VideoCapture(video_source)
        if not cap.isOpened():
            print("❌ Error: Could not open video source")
            return False
        fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        print(f"📊 Video properties: {width}x{height} @ {fps}fps")
        out = None
        if output_video:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))
        frame_count = 0
        start_time = time.time()
        print("\n🚀 Starting surveillance processing...")
        try:
            while True:
                ret, frame = cap.read()
                if not ret:
                    print("📺 End of video or camera disconnected")
                    break
                if max_frames and frame_count >= max_frames:
                    print(f"🎯 Reached maximum frames limit: {max_frames}")
                    break
                frame_count += 1
                self.detection_stats['total_frames'] = frame_count
                annotated_frame, results = self.process_frame(frame, frame_count)
                if display_frames and colab_mode:
                    try:
                        from google.colab.patches import cv2_imshow
                        if frame_count % 30 == 0:
                            print(f"📺 Frame {frame_count}:")
                            cv2_imshow(annotated_frame)
                    except ImportError:
                        print("⚠️ Could not import cv2_imshow, disabling frame display")
                if out:
                    out.write(annotated_frame)
                if results['alerts']:
                    for alert in results['alerts']:
                        print(f"🚨 ALERT: {alert['type']} detected for Person {alert['person_id']} "
                              f"(Frame {alert['frame']}, Confidence: {alert['confidence']:.2f})")
                if frame_count % 100 == 0:
                    elapsed_time = time.time() - start_time
                    fps_current = frame_count / elapsed_time
                    print(f"📊 Processing FPS: {fps_current:.1f}, Frame: {frame_count}")
        except Exception as e:
            print(f"❌ Error during processing: {e}")
            import traceback
            traceback.print_exc()
            return False
        finally:
            cap.release()
            if out:
                out.release()
            total_time = time.time() - start_time
            print(f"\n📈 FINAL STATISTICS:")
            print(f"   • Total Processing Time: {total_time:.1f} seconds")
            print(f"   • Frames Processed: {frame_count}")
            print(f"   • Violence Detected: {self.detection_stats['violence_detected']}")
            print(f"   • Normal Behavior: {self.detection_stats['normal_behavior']}")
            print(f"   • Abnormal Behavior: {self.detection_stats['abnormal_behavior']}")
            print("✅ Surveillance system shutdown complete")
            stats_path = os.path.join(self.drive_output_path, 'stats.json')
            try:
                with open(stats_path, 'w') as f:
                    json.dump(self.detection_stats, f)
                print(f"✅ Statistics saved at {stats_path}")
            except Exception as e:
                print(f"❌ Error saving stats.json to {stats_path}: {e}")
            return self.detection_stats['violence_detected']

# Process video if flag exists
process_flag = '/content/process_video.txt'
if os.path.exists(process_flag):
    with open(process_flag, 'r') as f:
        input_video_path = f.read().strip()
    if os.path.exists(input_video_path):
        output_video = os.path.join(DRIVE_OUTPUT_PATH, f"surveillance_output_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4")
        print(f"Processing video: {input_video_path}")
        # Copy input video to Google Drive
        input_video_drive_path = os.path.join(DRIVE_OUTPUT_PATH, os.path.basename(input_video_path))
        try:
            shutil.copy(input_video_path, input_video_drive_path)
            print(f"✅ Input video copied to {input_video_drive_path}")
        except Exception as e:
            print(f"⚠️ Error copying input video to {input_video_drive_path}: {e}")
        system = EnhancedRetailSurveillanceSystem(drive_output_path=DRIVE_OUTPUT_PATH)
        violence_detected = system.process_video(
            video_source=input_video_path,
            output_video=output_video,
            max_frames=100,
            colab_mode=True
        )
        if violence_detected is not False:
            result_text = 'Violence Detected' if violence_detected >= 1 else 'Normal - Violence Not Detected'
            print(f"📢 Result: {result_text}")
            result_path = os.path.join(DRIVE_OUTPUT_PATH, 'result.json')
            result_data = {'violence_detected': violence_detected}
            try:
                with open(result_path, 'w') as f:
                    json.dump(result_data, f)
                print(f"✅ Result saved at {result_path}")
            except Exception as e:
                print(f"❌ Error saving result.json: {e}")
        else:
            print("❌ Video processing failed. Check logs for details.")
        # Remove process flag
        os.remove(process_flag)
    else:
        print(f"❌ Video file not found: {input_video_path}")
else:
    print("No video to process. Upload a video in the Streamlit app and re-run this cell.")

In [ ]:
from pyngrok import ngrok
import time

# Terminate existing ngrok tunnels
ngrok.kill()

# Start ngrok tunnel
public_url = ngrok.connect(8501)
print(f"🌐 Streamlit app running at: {public_url}")

# Run Streamlit app with file watcher disabled
!streamlit run /content/surveillance_streamlit_app.py --server.port 8501 --server.address 0.0.0.0 --server.fileWatcherType none